### Set Seed

In [ ]:
def set_seed(seed_value=42):
    """Set seed for reproducibility for PyTorch and NumPy.

    Args:
        seed_value (int): The seed value to set for random number generators.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    # Additional steps for deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(42)  # You can replace 42 with any other seed value of your choice

### Create a Dataset class that returns the raw audio and labels (as tensors) in a single dictonary

In [4]:
from torch.utils.data import Dataset
import torch
import librosa
import pandas as pd
from transformers import AutoProcessor, AutoModelForCTC

#import wav2vec model and processor
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

class CustomAudioDataset(Dataset):
    def __init__(self, csv_path, processor):
        self.dataframe = pd.read_csv(csv_path)
        self.processor = processor

        # Extract column names for labels dynamically
        self.label_columns = list(self.dataframe.columns[2:])  #Exclude first two columns since these are irrelevant

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        audio_path = self.dataframe.iloc[idx]['mp3_path']
        
        # Select label columns based on the dynamically created list. This is grabbing all 188 class label names and converting to tensors.
        labels = torch.tensor(self.dataframe.iloc[idx][self.label_columns], dtype=torch.float32)

        # Load raw audio data using librosa
        audio_data, _ = librosa.load(audio_path, sr=16000)
        
        #Use processor to process audio file and return tensor of input values for model
        input_tensors = self.processor(audio_data, return_tensors="pt", sampling_rate=16000).input_values

        # Return a dictionary with input data and labels
        return {'input': input_tensors, 'labels': labels}

### Model Architecture. We first use the wav2vec autoencoder to generate audio embeddings. Then we add a few FF layers. Finally, we add a softmax layer for the output since we are doing multi-class classification.

In [ ]:
#This is just boilerplate code. We can update this if we want to make it deeper etc.
import torch.nn as nn

class CustomAudioModel(nn.Module):
    def __init__(self, ff_input_size=768, ff_output_size=64, num_classes=188):
        super(CustomAudioModel, self).__init__()

        # Define custom feed-forward layers
        self.fc1 = nn.Linear(ff_input_size, ff_output_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ff_output_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, embeddings):
        # Apply custom feed-forward layers directly to the extracted embeddings
        x = self.fc1(embeddings)
        x = self.relu(x)
        x = self.fc2(x)
        output = self.sigmoid(x)  # Apply sigmoid activation for multi-label classification

        return output

### Training module from Huggingface

In [ ]:
from transformers import Trainer, TrainingArguments
from torch.optim import Adam
import torch.nn.functional as F

# This needs to be updated with the appropriate values
model = CustomAudioModel(ff_input_size=768, ff_output_size=64, num_classes=188) #768 is the size of the embeddings from the wav2vec model

#These need to be updated
csv_path_train = 'data.csv'
csv_path_val = 'data.csv'

#Creating train and validate datasets
train_dataset = CustomAudioDataset(csv_path_train, processor)
val_dataset = CustomAudioDataset(csv_path_val, processor)

# Loss function for multi-label classification
def compute_loss(model, inputs):
    # Your custom loss calculation goes here
    logits = model(inputs['input'])
    loss = F.binary_cross_entropy_with_logits(logits, inputs['labels']) #appropiate loss function for multi-label classification
    return loss

# Optimizer
optimizer = Adam(model.parameters(), lr=.001)

# Training arguments -- these need to be adjusted
training_args = TrainingArguments(
    output_dir='./output_dir',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_total_limit=5,
    learning_rate=1e-4,
    save_steps=500,
    save_total_limit=5,
    gradient_accumulation_steps=1,
    compute_loss=compute_loss,
    optimizer=optimizer,
)

# Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

In [ ]:
# class CustomAudioModel(nn.Module):
#     def __init__(self, wav2vec_model_name="facebook/wav2vec2-base-960h", output_size=188, ff_output_size=64):
#         super(CustomAudioModel, self).__init__()

#         # Load the Wav2Vec 2.0 model and processor
#         self.wav2vec_model = AutoModelForCTC.from_pretrained(wav2vec_model_name)
#         self.processor = AutoProcessor.from_pretrained(wav2vec_model_name)

#         # Define custom feed-forward layers
#         self.fc1 = nn.Linear(768, ff_output_size)  # Adjust input size based on Wav2Vec 2.0 model's hidden size
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(ff_output_size, output_size)  # Adjust output size based on your task
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, audio_data):
#         # Process audio data using the Wav2Vec 2.0 model
#         input_tensors = self.processor(audio_data, return_tensors="pt", sampling_rate=16000).input_values
#         with torch.no_grad():
#             embeddings = self.wav2vec_model(input_tensors).last_hidden_state.mean(dim=1)

#         # Apply custom feed-forward layers
#         x = self.fc1(embeddings)
#         x = self.relu(x)
#         x = self.fc2(x)
#         output = self.softmax(x)  # Apply sigmoid activation for multi-label classification

#         return output

#Instantiate the model
# model = CustomAudioModel()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# loss_fn = nn.CrossEntropyLoss()
# num_epochs = 3

# audio_data = torch.randn(1, 16000)  # Replace with your actual audio data
# output = model(audio_data)
# print(output.shape)  # This will be (batch_size, output_size), where output_size is 188 in your case


### Train the model

In [ ]:
# for epoch in range(num_epochs):
#     for batch in data_loader:
#         inputs = batch['input']
#         labels = batch['labels']

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(inputs)

#         # Compute the loss
#         loss = loss_fn(outputs, labels)

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


#### Processing the train files

In [ ]:
# import os
# from datasets import load_dataset
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoProcessor, AutoModelForCTC
# import torch
# import librosa
# import pandas as pd

# # Load your dataset from the CSV file
# csv_path = 'path/to/your/csv/file.csv'
# df = pd.read_csv(csv_path)

# # Load pretrained model and processor
# model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
# processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

# class CustomAudioDataset(Dataset):
#     def __init__(self, dataframe, processor):
#         self.dataframe = dataframe
#         self.processor = processor

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         audio_path = self.dataframe.iloc[idx]['audio_file']
#         labels = self.dataframe.iloc[idx]['labels']

#         # Load audio file and process using the Wav2Vec processor
#         audio_data, _ = librosa.load(audio_path, sr=16000)
#         input_tensors = self.processor(audio_data, return_tensors="pt", sampling_rate=16000).input_values

#         return {'input': input_tensors, 'labels': torch.tensor(labels, dtype=torch.float32)}

# # Create an instance of your custom dataset
# audio_dataset = CustomAudioDataset(df, processor)

# # Create a PyTorch DataLoader for batching and shuffling
# batch_size = 32  # Adjust as needed
# data_loader = DataLoader(audio_dataset, batch_size=batch_size, shuffle=True)

# # Example usage in a training loop
# for batch in data_loader:
#     inputs = batch['input']
#     labels = batch['labels']

    # Forward pass, loss calculation, backward pass, optimization, etc.
    # Your training code goes here


In [3]:
# #Path to the directory containing the audio files
# train_audio = 'train'

# #list all files in the directory
# audio_files = [os.path.join(train_audio, file) for file in os.listdir(train_audio)]

# #Define number of files to process
# num_files = 10

# #iterate over audio file and extract embeddings.

# for i, audio_file in enumerate(audio_files):
#   embeddings = extract_audio_embeddings(audio_file)
  
#   #save embeddings in a numpy array
#   if i == 0:
#     embeddings_array = embeddings
#   else:
#     embeddings_array = np.vstack((embeddings_array, embeddings))
    
#   #Check if number of files to process has been reached
#   if i + 1 == num_files:
#     print(f'Processed {num_files} files. Stopping the iteration.')
#     break

AttributeError: 'CausalLMOutput' object has no attribute 'last_hidden_state'